In [14]:
# Pandas
import pandas as pd

# Prophet
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric

# Prophet evaluation
from prophet.diagnostics import cross_validation, performance_metrics

# Plotting
import plotly.express as px

from datetime import datetime, timezone, time, timedelta, date

# Introducción


En este notebook vamos a ver un ejemplo de cómo sería un ejemplo end-to-end aplicando el dataset de los pasajeros



# Análisis exploratorio descriptivo

El análisis exploratorio busca ofrecer una panorámica general de los datos disponibles que componen la serie temporal y ayudar a identificar patrones temporales que podrían introducirse en el modelo.. Para ello, es necesario entender bien el contexto de negocio en el que se hace el análisis de series temporales. Una buena estrategia para hacer un buen análisis descriptivo es hacer preguntas que nos ayuden detectar posibles características que tengamos que incluir en el pre-procesado.



In [15]:
pass_df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv', parse_dates=['Month'])
pass_df

,Month,Passengers
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121
...,...,...
139,1960-08-01,606
140,1960-09-01,508
141,1960-10-01,461
142,1960-11-01,390


In [16]:
# Esta librería permite sacar estadísticas descriptivas de nuestros datos
# Da info también de los 'missing' si los tuvieramos (que no es el caso)
import skimpy as sk
sk.skim(pass_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 144    │ │ datetime64  │ 1     │                                                          │
│ │ Number of columns │ 2      │ │ int32       │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name       ┃ NA   ┃ NA %    ┃ mean    ┃ sd     ┃ p0    ┃ p25   ┃ p50   ┃ p75   ┃ p100    ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩  │
│ │ Passengers        │    0 │       0 │     280 │    120 │   100 │   180 │   270 │   360 │     620 │  ▇▇▆▃▂▁  │  │
│ └───────────────────┴──────┴─────────┴─────────┴────────┴───────┴───────┴───────┴───────┴─────────┴──────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name            ┃ NA     ┃ NA %      ┃ first               ┃ last                ┃ frequency        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ Month                  │      0 │         0 │     1949-01-01      │     1960-12-01      │ MS               │  │
│ └────────────────────────┴────────┴───────────┴─────────────────────┴─────────────────────┴──────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

**¿Qué podemos decir de los datos con la información que nos da skimpy?**

- Tenemos una muestra de 144 observaciones mensuales en total para el período que va desde 1949 hasta 1960.

- Observamos, en media, 280 pasajeros mensuales. El mes con menos pasajeros, hubo 100 mientras que mes con más pasajeros hubo 620. 

- No se observa, a priori, presencia de períodos anómalos 


**¿Cómo evoluciona en el tiempo la serie temporal?**

- Se aprecia una tendencia positiva a lo largo del tiempo que va a requerir un análisis de estacionariedad

- Asimismo, hay un patrón de estacionalidad a partir del cual los meses de verano (Julio - Agosto) experimentan aumentos en la demanda. Esto puede estar asociado al hecho de que estos meses son periodos vacacionales. 


In [17]:
px.line(pass_df, x ='Month', y = 'Passengers', title = 'Pasajeros mensuales | 1949 -  1960')

**¿Cuál es la diferencia de demanda para cada uno de los años? ¿Y para cada unos de los meses?** 

- Considerando el año inicial como referencia, el número de pasajeros se incrementa en un 276% aproximadamente durante el último año 

- Hay un incremento de pasajeros generalizado a lo largo de la serie temporal. No obstante, hay años como 1956 ó 1958,  donde el crecimiento con resepecto al año anterior es bastante reducido (6% y 3% respectivamente)  en comparación con otros con otros años como 1951 (22%) o 1955 (19%)

- Los meses de verano, Julio y Agosto, son los que presentan una mayor demanda y una mayor distribución de valores. Noviembre, Enero y Febrero son los meses con menos demanda con una mediana entorno a 220 pasajeros. 


In [18]:
# Definimos el mes como índice
# Es interesante hacerlo porque nos permite sacar las características más rápido
pass_df.set_index('Month', inplace=True)

In [19]:
# 1. Extrae información de meses y años 

pass_df['year'] = pass_df.index.year
pass_df['month_name']= pass_df.index.month_name()
pass_df.head(2)


,Passengers,year,month_name
Month,,,
1949-01-01,112,1949,January
1949-02-01,118,1949,February


In [20]:
# Agrupa por año y suma para ver la demanda agregada por año y cómo crece

grouped = pd.DataFrame(pass_df.groupby('year')['Passengers'].sum())

px.bar(grouped, x = grouped.index, y = 'Passengers', text = 'Passengers',  title = 'Pasajeros totales por año')


In [21]:
# Evaluar el cambio porcentual
grouped['pct_change'] = grouped.pct_change().round(2)*100

In [22]:
px.line(grouped, x = grouped.index, y = 'pct_change',  title = 'Crecimiento porcentual de pasajeros por año')

En esta gráfica ves el porcentaje de crecimiento con respecto al año anterior. En el 1954 y 1958 ves que hay una caída del crecimiento con respecto a otros años.

In [23]:
# Agrupa por mes para ver la dispersión de los datos
# Creamos una variable categórica con los meses
mes_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October','November','December']
pass_df['month_name'] = pd.Categorical(pass_df['month_name'] , categories=mes_order, ordered=True)

# Plot
fig = px.box(pass_df, x='month_name', y='Passengers', labels={'month_name': 'Month', 'Passengers': 'Passengers'})
fig.show()

Vemos un mayor número de pasajeros en los meses de verano en media que en los meses de invierno, y también la dispersión de los datos es mayor.

# El modelo: Prophet


Prophet es un algoritmo / procedimiento de predicción de datos de series temporales basado en un modelo aditivo en el que se ajustan las tendencias no lineales para la estacionalidad anual, semanal y diaria, además de los efectos de días festivos. Prophet funciona mejor con series temporales que tienen efectos estacionales fuertes (ya sean semanales o mensuales) y una cantidad suficiente de datos históricos donde se representa esa estacionalidad. Prophet es robusto ante datos faltantes y cambios de tendencia, y tiende a manejar bien los valores atípicos.

Este software de código abierto fue publicado por el equipo de Core Data Science de Facebook. Está disponible para su descarga en CRAN (R) y PyPI (Python).

Prophet es un modelo aditivo de regresión que incluye varios métodos avanzados e inteligentes de pronóstico, incluido el análisis de puntos de cambio:
+ Una tendencia de crecimiento lineal o logístico en segmentos. Prophet detecta automáticamente cambios en las tendencias seleccionando puntos de cambio en los datos.
+ Un componente estacional anual modelado mediante series de Fourier.
+ Un componente estacional semanal mediante variables ficticias.
+ Una lista proporcionada por el usuario de días festivos importantes.

Comenzaremos utilizando la serie de tiempo de Pasajeros para mostrar un modelo básico de Prophet.

## Manipulación de los datos

Los datos de entrada para Prophet siempre siguen el mismo formato con dos columnas: ds y y.

- La columna `ds` (fecha o marca de tiempo) debe estar en el formato esperado por Pandas, idealmente YYYY-MM-DD para una fecha o YYYY-MM-DD HH:MM:SS para una marca de tiempo.

- La columna `y` debe ser numérica y representa el valor que queremos pronosticar.

El análisis descriptivo debería proporcionar una comprensión de cómo se ve la serie temporal y ayudar a identificar patrones temporales que podrían introducirse en el modelo.

In [24]:
# Construímos df para modelizar
# Si hubiera más variables que nos interesan para el modelo, habría que meteralas en una columna llamada 'x'
# Cambiamos el nombre de los datos, ajustando a ds e y
pass_prophet = pass_df[['Passengers']]
pass_prophet = pass_prophet.rename(columns = {'Passengers':'y'})
# La variable fecha tiene que ser ds además de un índice
pass_prophet['ds'] = pass_prophet.index
pass_prophet.reset_index(drop=True, inplace=False)
display(pass_prophet.head(2))

display(pass_prophet.info())

,y,ds
Month,,
1949-01-01,112,1949-01-01
1949-02-01,118,1949-02-01


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 144 entries, 1949-01-01 to 1960-12-01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   y       144 non-null    int64         
 1   ds      144 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.4 KB


None

## Ajuste del modelo

In [25]:
m = Prophet()
m.fit(pass_prophet)

AttributeError: 'Prophet' object has no attribute 'stan_backend'

## Proyecciones

Primero, necesitamos generar el conjunto de datos con las fechas que queremos predecir. Estas fechas se incluirán en una columna llamada `ds` y las predicciones se agregarán a esta columna. El marco de datos se crea utilizando el método `Prophet.make_future_dataframe()`. De manera predeterminada, también incluirá las fechas históricas con las que entrenamos el modelo, por lo que podremos ver la calidad del ajuste del modelo.

In [26]:
# Proyectar a un año vista
# Creamos un df para almacenar las predicciones que hagamos, en este caso un año más. 

future_df = m.make_future_dataframe(periods=12, freq='MS') # make an extra year
future_df

NameError: name 'm' is not defined

In [ ]:
# Utilizando ese df anterior, corremos la función predict y nos saldrá toda esa información
forecast_df = m.predict(future_df)

display(forecast_df.head(5))
display(forecast_df.tail(5))

In [ ]:
# Sacamos la fecha, 'ythat' predicción, 'ythat_lower' e 'ythat_upper' intervalo de confianza por la izquierda y por la derecha.
forecast_clean = forecast_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_clean

 ## Representar los resultados

In [ ]:
# Representamos las predicciones del modelo
# Los botoncitos de arriba(el primero coge la info semanal, segundo info mensual, tercero 6 meses, cuarto todo el año)
# En este ejemplo utilizamos la predicción fuera de muestra
plot_plotly(m, forecast_df)

 ## Descomposición de los resultados

Si deseamos ver los componentes del pronóstico, podemos utilizar el método `Prophet.plot_components()`. De manera predeterminada, se mostrarán la tendencia y las estacionalidades de la serie temporal. Si se incluyen días festivos, también aparecerán en el gráfico.

In [ ]:
# Descomponemos la predicción
plot_components_plotly(m, forecast_df)

## Evaluación de las predicciones

Vamos a comparar las predicciones en la muestra y fuera de la muestra

In [ ]:
# División en train y test
# Dos años y medio en test 
train_df = pass_prophet.loc[pass_prophet['ds'] < '1957-06-01']
test_df = pass_prophet.loc[pass_prophet['ds'] >= '1957-06-01']

In [ ]:
model_ml = Prophet(interval_width = 0.9, weekly_seasonality = True)
model_ml.fit(train_df) # Si años otras variables sería train df['y], train['x'])

In [ ]:
# Proyectar a un año vista (fuera de la muestra)
# Le decimos que considere el período de test + 12 meses más
future_ml_df = model_ml.make_future_dataframe(periods =len(test_df)+12, freq = 'MS')
future_ml_df

In [ ]:
# Corremos el predict y seleccionar las variables que estés interesado
forecast_ml_df = model_ml.predict(future_ml_df)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_ml_df

In [ ]:
# Concatenate datasets to compare in-and-out samples
# Vemos observados con respecto a lo que estamos prediciendo

px.line(pd.concat([pass_prophet.set_index('ds')['y'], forecast_ml_df.set_index('ds')['yhat']], axis = 1), title = 'Proyecciones con Prophet | Estacionalidad aditiva')

## Incluyendo más complejidad...

Como vimos en la primera sesión, la estacionalidad podía seguir distintas especificaciones. En este caso concreto, vimos que el patrón estacional se correspondía con un modelo multiplicativo. Añadimos a la instancia de Prophet este tipo estacionalidad

## Estacionalidad multiplicativa

In [ ]:
# Metemos un carácter multiplicativo que optimiza mejor el modelo. Antes lo hemos hecho de forma aditiva.
m_pax_mul = Prophet(seasonality_mode='multiplicative', weekly_seasonality=True)
# Utilizo todo el dataset, pero podría dividirlo en train y test
m_pax_mul.fit(pass_prophet)

In [ ]:
future_pax_mul = m_pax_mul.make_future_dataframe(12, freq='MS')
display(future_pax_mul.head(10))
display(future_pax_mul.tail(10))

In [ ]:
# Vemos que ahora se ajusta mejor
forecast_pax_mul = m_pax_mul.predict(future_pax_mul)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
px.line(pd.concat([pass_prophet.set_index('ds')['y'], forecast_pax_mul.set_index('ds')['yhat']], axis = 1), title = 'Proyecciones con Prophet - Estacionalidad multiplicativa')

In [ ]:
# Vemos las predicciones fuera de la muestra en valores
# Comparar los valores que hemos estimado: valor medio, varianza...
sk.skim(forecast_pax_mul.loc[forecast_pax_mul['ds'] >= '1961-01-01'])


## Evaluación

###  Validación cruzada

Utilizamos validación cruzada para evaluar los resultado tanto del modelo con estacionalidad aditiva como de estacionalidad multiplicativa. 



In [ ]:

# Source: https://facebook.github.io/prophet/docs/diagnostics.html

# Definimos initial (muestra de train en días(tamaño)), a partir de esa se incrementa 180 días, y quieres que la predicción se de un año(tiene que ser expresada en días)
# Cross validation para modelo aditivo
cv_add = cross_validation(model = model_ml, initial='730 days', period='180 days', horizon = '365 days')
# Todos los resultado de tu evaluación lo utilizas para predecir esos 365 días
# Cada incremento de 180 lo coge de test
# Estamos metiendo cross validation a los dos modelos. Evaluarlos bajo los mismos criterios(partición,etc)
# - pick 730 days of training data initially
# - make predictions every 180 days
# - evaluate the prediction performance on an horizon of 365 days

In [ ]:
# Cross validation para modelo multiplicativo
cv_mul = cross_validation(model = m_pax_mul,  initial='730 days', period='180 days', horizon = '365 days' )

In [ ]:
# Df de métricas del aditivo
df_performance_add = performance_metrics(cv_add)
display(df_performance_add.head(10))

In [ ]:
# Df de métricas del multiplicativo. Vemos que los errores son menores que en el aditivo
df_performance_mul = performance_metrics(cv_mul)
display(df_performance_mul.head(10))

In [ ]:
# Pintamos los errores del aditivo
cv_plot = plot_cross_validation_metric(cv_add, metric = 'rmse')

In [ ]:
# Pintamos los errores del multiplicativo. Tienen un comportamiento más compacto y regular. Por tanto, vemos que funciona mejor.
cv_plot_mul = plot_cross_validation_metric(cv_mul, metric = 'rmse')

In [ ]:
df_performance_add.describe().transpose()

In [ ]:
df_performance_mul.describe().transpose()

# Conclusiones

- Se han evaluado dos especificaciones distintas del modelo: una especificación que ha añadido la estacionalidad de carácter aditivo y una de carácter multiplicativo. 

- Como se puede observar a partir de los gráficos de la proyección con cada una de las estacionalidades, el modelo con estacionalidad multiplicativa presenta un mejor ajuste a las muestras de entrenamiento y evaluación. 

- Asimismo, este resultado se confirma para pronósticos fuera de la muestra y también con los resultados derivados de la evaluación cruzada donde la media de todas las métricas de error consideradas disminuye con respecto a los resultados con estacionalidad aditiva. Asimismo, estos resultados presentan una menor dispersión (varianza) lo que apunta a una menor incertidumbre en el resultado.



##  Más complejidad, añadiendo particularidades del calendario.


Vamos a ver otro ejemplo: la serie temporal del registro de visitas diarias a la página de Wikipedia de Peyton Manning. Esta serie es un buen ejemplo porque ilustra algunas de las características de Prophet, así como permite agregar más complejidades al modelo, como la estacionalidad múltiple (semanal, mensual), días festivos o la capacidad de modelar días especiales (como las apariciones de Manning en los playoffs y el Super Bowl).

## Análisis descriptivo

In [27]:
# Read the data
link = 'https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_peyton_manning.csv'
peyton_df = pd.read_csv(link)
peyton_df.head()
# Viene ya con formato de Prophet

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


In [28]:
peyton_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ds      2905 non-null   object 
 1   y       2905 non-null   float64
dtypes: float64(1), object(1)
memory usage: 45.5+ KB


In [29]:
# Cambiamos el tipo a datetime
peyton_df['ds'] = pd.to_datetime(peyton_df['ds'])
peyton_df.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
ds,2905,2012-01-15 10:11:11.669535232,2007-12-10 00:00:00,2010-01-12 00:00:00,2012-01-23 00:00:00,2014-01-21 00:00:00,2016-01-20 00:00:00,NaN
y,2905.0,8.138958,5.26269,7.5148,7.997999,8.580168,12.846747,0.845957


In [30]:
# De septiembre a diciembre más visitas al fútbol americano
px.line(peyton_df, x ='ds', y = 'y', title = 'Visitas a la web de Peyton Manning')

## Modelo

### Ajuste del modelo

In [31]:
# Modelo simple sin estacionalidad
# No hay mensuales
m_peyton = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=True)
m_peyton.fit(peyton_df)

AttributeError: 'Prophet' object has no attribute 'stan_backend'

### Proyecciones

In [ ]:
# Make dataframe for the future
# Definimos el período a predecir
peyton_future = m_peyton.make_future_dataframe(periods=365)

peyton_future.tail()

In [ ]:
# Make forecasts
forecast_peyton = m_peyton.predict(peyton_future)
forecast_peyton

In [ ]:
# Plot in and out sample forecasts
plot_plotly(m_peyton, forecast_peyton)

Captura la tendencia, en eventos más extraordinarios se pierde.

### Descomposición de la serie temporal

In [32]:
# Decomposing the time series

plot_components_plotly(m_peyton, forecast_peyton)

NameError: name 'm_peyton' is not defined

### Evaluación cruzada

In [ ]:
# Cross validation

cv_peyton = cross_validation(model = m_peyton, initial='730 days', period='180 days', horizon = '365 days')
# Utilizo los mismo datos que en el df de pasajeros
# - pick 730 days of training data initially
# - make predictions every 180 days
# - evaluate the prediction performance on an horizon of 365 days


In [ ]:
# Sacamos métricas y variables descriptivas
peyton_performance = performance_metrics(cv_peyton)
peyton_performance.describe().transpose()

## Añadiendo vacaciones

In [ ]:
# Añadimos a los datos las vaciones de ee.uu. Festivos nacionales
peyton_hols = Prophet().add_country_holidays(country_name='US')
peyton_hols.fit(peyton_df)

In [ ]:
# What holidays are included

peyton_hols.train_holiday_names

In [ ]:
# Add customised holidays
# Se tienen que llamar así 'holiday', 'ds', 'lower_window','upper_window'
playoffs = pd.DataFrame({
    'holiday': 'playoff',#se colocan buscando la info
    'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                          '2010-01-24', '2010-02-07', '2011-01-08',
                          '2013-01-12', '2014-01-12', '2014-01-19',
                          '2014-02-02', '2015-01-11', '2016-01-17',
                          '2016-01-24', '2016-02-07']), # Demás eventos extraordinarios. Especificar datetime
    'lower_window': 0,#siempre va con estos valores
    'upper_window': 1,
})

superbowls = pd.DataFrame({
    'holiday': 'superbowl',
    'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
    'lower_window': 0, # igual que arriba
    'upper_window': 1,
})

holidays_custom = pd.concat((playoffs, superbowls))

holidays_custom.head()


In [ ]:
# Adding customised holiday periods
# Se mete siempre en el argumento holidays
peyton_hols_augmented = Prophet(holidays = holidays_custom).add_country_holidays(country_name='US')
peyton_hols_augmented.fit(peyton_df)

In [ ]:
# Make dataframe for the future
peyton_future_hols = peyton_hols_augmented.make_future_dataframe(periods=365)

# Make forecasts
forecast_peyton_hols = peyton_hols_augmented.predict(peyton_future_hols)

forecast_peyton_hols

In [ ]:
# Plot in and out sample forecasts
plot_plotly(peyton_hols_augmented, forecast_peyton_hols)

In [ ]:
# Decompose

plot_components_plotly(peyton_hols_augmented, forecast_peyton_hols)

### Comparación entre vacaciones y no vacaciones

In [ ]:
# Cross validation
cv_peyton_hols = cross_validation(model = peyton_hols_augmented, initial='730 days', period='180 days', horizon = '365 days')

In [ ]:
peyton_hols_performance = performance_metrics(cv_peyton_hols)

display(f'No hols Prophet model ----------------------', peyton_performance.describe().transpose())
display(f'Hols model Prophet model ------------------- ', peyton_hols_performance.describe().transpose())